In [1]:
import pandas as pd, numpy as np, matplotlib.pyplot as plt, matplotlib as mpl, seaborn as sns, warnings

%matplotlib inline
from matplotlib import font_manager, rc
font_name = font_manager.FontProperties(fname="c:/Windows/Fonts/malgun.ttf").get_name()
plt.rc('font', family='NanumGothic')
rc('font', family=font_name)
mpl.rcParams['axes.unicode_minus'] = False

plt.style.use('ggplot')
color_pal = ["#F8766D", "#D39200", "#93AA00", "#00BA38", "#00C19F", "#00B9E3", "#619CFF", "#DB72FB"]

warnings.filterwarnings(action='ignore')

from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import StackingRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.svm import LinearSVR, SVR
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import MinMaxScaler, StandardScaler, LabelEncoder
from sklearn.model_selection import train_test_split

In [2]:
df_raw = pd.read_csv('./data/jeju_data_ver1/201901-202003.csv')

In [13]:
df_raw.head()

,REG_YYMM,CARD_SIDO_NM,CARD_CCG_NM,STD_CLSS_NM,HOM_SIDO_NM,HOM_CCG_NM,AGE,SEX_CTGO_CD,FLC,CSTMR_CNT,AMT,CNT
0,201901,강원,강릉시,건강보조식품 소매업,강원,강릉시,20s,1,1,4,311200,4
1,201901,강원,강릉시,건강보조식품 소매업,강원,강릉시,30s,1,2,7,1374500,8
2,201901,강원,강릉시,건강보조식품 소매업,강원,강릉시,30s,2,2,6,818700,6
3,201901,강원,강릉시,건강보조식품 소매업,강원,강릉시,40s,1,3,4,1717000,5
4,201901,강원,강릉시,건강보조식품 소매업,강원,강릉시,40s,1,4,3,1047300,3


In [14]:
df_raw.columns

Index(['REG_YYMM', 'CARD_SIDO_NM', 'CARD_CCG_NM', 'STD_CLSS_NM', 'HOM_SIDO_NM',
       'HOM_CCG_NM', 'AGE', 'SEX_CTGO_CD', 'FLC', 'CSTMR_CNT', 'AMT', 'CNT'],
      dtype='object')

#### 1, 2, 3월 데이터만 추출

In [15]:
df = df_raw[(df_raw['REG_YYMM'] == 202001) | (df_raw['REG_YYMM'] == 202002) | (df_raw['REG_YYMM'] == 202003)]
df.head()

,REG_YYMM,CARD_SIDO_NM,CARD_CCG_NM,STD_CLSS_NM,HOM_SIDO_NM,HOM_CCG_NM,AGE,SEX_CTGO_CD,FLC,CSTMR_CNT,AMT,CNT
20425415,202001,강원,강릉시,건강보조식품 소매업,강원,강릉시,20s,2,1,3,345000,3
20425416,202001,강원,강릉시,건강보조식품 소매업,강원,강릉시,30s,1,2,3,1903450,3
20425417,202001,강원,강릉시,건강보조식품 소매업,강원,강릉시,30s,2,2,14,1520500,15
20425418,202001,강원,강릉시,건강보조식품 소매업,강원,강릉시,40s,1,3,9,1239200,9
20425419,202001,강원,강릉시,건강보조식품 소매업,강원,강릉시,40s,1,4,3,606700,4


In [16]:
df.fillna('세종시', inplace=True)

In [17]:
df.columns

Index(['REG_YYMM', 'CARD_SIDO_NM', 'CARD_CCG_NM', 'STD_CLSS_NM', 'HOM_SIDO_NM',
       'HOM_CCG_NM', 'AGE', 'SEX_CTGO_CD', 'FLC', 'CSTMR_CNT', 'AMT', 'CNT'],
      dtype='object')

In [18]:
df.drop(['CARD_CCG_NM','HOM_SIDO_NM','HOM_CCG_NM'], axis=1, inplace=True)

In [19]:
df = df.groupby(['REG_YYMM', 'CARD_SIDO_NM', 'STD_CLSS_NM', 'AGE', 'SEX_CTGO_CD', 'FLC']).sum().reset_index()

In [20]:
df.head()

,REG_YYMM,CARD_SIDO_NM,STD_CLSS_NM,AGE,SEX_CTGO_CD,FLC,CSTMR_CNT,AMT,CNT
0,202001,강원,건강보조식품 소매업,20s,1,1,6,168500,6
1,202001,강원,건강보조식품 소매업,20s,2,1,13,764646,13
2,202001,강원,건강보조식품 소매업,30s,1,2,72,12017410,73
3,202001,강원,건강보조식품 소매업,30s,2,2,88,9867625,89
4,202001,강원,건강보조식품 소매업,40s,1,3,90,10645775,91


# 군집 데이터 불러오기

In [21]:
cluster_info = pd.read_csv("./data/class_cluster.txt")

In [22]:
for i in cluster_info.iloc[2]:
    print(i)

['택시 운송업']


# 3번 군집

In [23]:
class_list = ['택시 운송업']

In [24]:
df = df[df['STD_CLSS_NM'].map(lambda x: x in class_list)]

In [25]:
df.head()

,REG_YYMM,CARD_SIDO_NM,STD_CLSS_NM,AGE,SEX_CTGO_CD,FLC,CSTMR_CNT,AMT,CNT
659,202001,강원,택시 운송업,20s,1,1,12,110540,15
660,202001,강원,택시 운송업,20s,2,1,4,17500,5
661,202001,강원,택시 운송업,50s,1,4,5,27300,5
662,202001,강원,택시 운송업,60s,1,5,3,35000,3
663,202001,강원,택시 운송업,60s,2,5,3,14900,3


In [26]:
df.STD_CLSS_NM.unique()

array(['택시 운송업'], dtype=object)

In [27]:
def grap_year(data):
    data = str(data)
    return int(data[:4])

def grap_month(data):
    data = str(data)
    return int(data[4:])

In [28]:
df['year'] = df['REG_YYMM'].apply(lambda x: grap_year(x))
df['month'] = df['REG_YYMM'].apply(lambda x: grap_month(x))
df = df.drop(['REG_YYMM'], axis=1)

# Label Encoding

In [29]:
# 인코딩
dtypes = df.dtypes
encoders = {}
for column in df.columns:
    if str(dtypes[column]) == 'object':
        encoder = LabelEncoder()
        encoder.fit(df[column])
        encoders[column] = encoder
        
df = df.copy()        
for column in encoders.keys():
    encoder = encoders[column]
    df[column] = encoder.transform(df[column])

In [30]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 552 entries, 659 to 38414
Data columns (total 10 columns):
 #   Column        Non-Null Count  Dtype
---  ------        --------------  -----
 0   CARD_SIDO_NM  552 non-null    int32
 1   STD_CLSS_NM   552 non-null    int32
 2   AGE           552 non-null    int32
 3   SEX_CTGO_CD   552 non-null    int64
 4   FLC           552 non-null    int64
 5   CSTMR_CNT     552 non-null    int64
 6   AMT           552 non-null    int64
 7   CNT           552 non-null    int64
 8   year          552 non-null    int64
 9   month         552 non-null    int64
dtypes: int32(3), int64(7)
memory usage: 41.0 KB


In [31]:
df.STD_CLSS_NM.unique()

array([0])

# Preprocessing

In [32]:
# 변수명 지정
X_cols = list(df.columns)
X_cols.remove('AMT')

In [33]:
X = df.drop(['CSTMR_CNT','AMT','CNT'], axis=1)
y = np.log1p(df['AMT'])

In [34]:
# X 변수 Scale 적용
# scaler = StandardScaler()
# X = scaler.fit_transform(X)

In [35]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, shuffle=True)

In [36]:
X_train

,CARD_SIDO_NM,STD_CLSS_NM,AGE,SEX_CTGO_CD,FLC,year,month
27600,1,0,4,2,4,2020,3
27594,1,0,3,2,2,2020,3
6183,6,0,1,1,2,2020,1
16335,3,0,3,2,3,2020,2
27583,1,0,1,2,1,2020,3
...,...,...,...,...,...,...,...
1574,1,0,2,1,1,2020,1
14844,1,0,5,1,5,2020,2
4653,5,0,2,2,2,2020,1
3141,3,0,1,2,1,2020,1


# Light Gradient Boosting Machine

In [37]:
import lightgbm as lgb

In [38]:
train_ds = lgb.Dataset(X_train, label=y_train)
val_ds = lgb.Dataset(X_test, label=y_test)

In [39]:
params = {
            'learning_rate' : 0.1,
            'boosting_type': 'gbdt',
            'objective': 'tweedie',
            'tweedie_variance_power': 1.1,
            'metric': 'rmse',
            'sub_row' : 0.75,
            'lambda_l2' : 0.1
        }

In [40]:
model = lgb.train(params,
                  train_ds,
                  5000,
                  val_ds,
                  verbose_eval = 100,
                  early_stopping_rounds = 100
                 )

Training until validation scores don't improve for 100 rounds
[100]	valid_0's rmse: 1.59444
[200]	valid_0's rmse: 1.56457
[300]	valid_0's rmse: 1.41041
[400]	valid_0's rmse: 1.39573
[500]	valid_0's rmse: 1.39923
Early stopping, best iteration is:
[434]	valid_0's rmse: 1.39269


In [41]:
X.columns

Index(['CARD_SIDO_NM', 'STD_CLSS_NM', 'AGE', 'SEX_CTGO_CD', 'FLC', 'year',
       'month'],
      dtype='object')

In [42]:
# 예측 템플릿 만들기
CARD_SIDO_NMs = df['CARD_SIDO_NM'].unique()
STD_CLSS_NMs  = df['STD_CLSS_NM'].unique()
#CARD_CCG_NMs  = df['CARD_CCG_NM'].unique()
AGEs          = df['AGE'].unique()
SEX_CTGO_CDs  = df['SEX_CTGO_CD'].unique()
FLCs          = df['FLC'].unique()
years         = [2020]
months        = [4]

temp = []
for CARD_SIDO_NM in CARD_SIDO_NMs:
    for STD_CLSS_NM in STD_CLSS_NMs:
        for AGE in AGEs:
            for SEX_CTGO_CD in SEX_CTGO_CDs:
                for FLC in FLCs:
                    for year in years:
                        for month in months:
                            temp.append([CARD_SIDO_NM, STD_CLSS_NM,  AGE, SEX_CTGO_CD, FLC, year, month])
temp = np.array(temp)
temp = pd.DataFrame(data=temp, columns=X.columns)

In [43]:
X.columns

Index(['CARD_SIDO_NM', 'STD_CLSS_NM', 'AGE', 'SEX_CTGO_CD', 'FLC', 'year',
       'month'],
      dtype='object')

In [44]:
temp

,CARD_SIDO_NM,STD_CLSS_NM,AGE,SEX_CTGO_CD,FLC,year,month
0,0,0,1,1,1,2020,4
1,0,0,1,1,4,2020,4
2,0,0,1,1,5,2020,4
3,0,0,1,1,2,2020,4
4,0,0,1,1,3,2020,4
...,...,...,...,...,...,...,...
905,8,0,6,2,1,2020,4
906,8,0,6,2,4,2020,4
907,8,0,6,2,5,2020,4
908,8,0,6,2,2,2020,4


In [45]:
# 예측
pred = model.predict(temp)

In [46]:
pred = np.expm1(pred)
temp['AMT'] = np.round(pred, 0)
temp['REG_YYMM'] = temp['year']*100 + temp['month']
temp = temp[['REG_YYMM', 'CARD_SIDO_NM', 'STD_CLSS_NM', 'AMT']]
temp = temp.groupby(['REG_YYMM', 'CARD_SIDO_NM', 'STD_CLSS_NM']).sum().reset_index(drop=False)

In [47]:
temp

,REG_YYMM,CARD_SIDO_NM,STD_CLSS_NM,AMT
0,202004,0,0,8.752430e+07
1,202004,1,0,8.752430e+07
2,202004,2,0,1.396018e+07
3,202004,3,0,2.136618e+06
4,202004,4,0,3.248872e+09
5,202004,5,0,3.248872e+09
6,202004,6,0,3.285111e+09
7,202004,7,0,1.492798e+11
8,202004,8,0,7.405285e+07
9,202004,9,0,7.405285e+07


In [48]:
encoders

{'CARD_SIDO_NM': LabelEncoder(),
 'STD_CLSS_NM': LabelEncoder(),
 'AGE': LabelEncoder()}

In [49]:
# 디코딩 
temp['CARD_SIDO_NM'] = encoders['CARD_SIDO_NM'].inverse_transform(temp['CARD_SIDO_NM'])
temp['STD_CLSS_NM'] = encoders['STD_CLSS_NM'].inverse_transform(temp['STD_CLSS_NM'])

In [50]:
temp.to_csv('./data/clusters/cluster3.csv')